In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk

import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import make_blobs, make_circles

from sklearn.cluster import KMeans

##### Synthetic data with different range

In [ ]:
blob_centers = np.array(
    [[ 11.2,  6.3],
     [-1.0 ,  6.3],
     [-4.7,  7.8],
     [-8.0,  12.8],
     [-12.5,  13.3]])

blob_std = np.array([0.4, 0.3, 0.6, 0.1, 0.2])

In [ ]:
X, y = make_blobs(n_samples=300, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [ ]:
def plot_clusters(X, y=None):
    #plt.scatter(X[:, 0], X[:, 1], c=y, s=1)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=1, cmap='rainbow')
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0)

In [ ]:
plot_clusters(X, y)

In [ ]:
##### Synthetic data for KMeans

In [ ]:
num_clusters = 4
X, y = make_blobs(n_samples=300, centers=num_clusters, cluster_std=0.6, random_state=0)
plot_clusters(X, y)

In [ ]:
kmeans = KMeans(n_clusters=num_clusters, random_state=42, verbose=1)
kmeans.fit(X)

In [ ]:
y_pred = kmeans.predict(X)

print("X[i] belongs to cluster:\n", y_pred)

In [ ]:
print("Instance labels:\n", kmeans.labels_)
print("Cluster Centroids:\n", kmeans.cluster_centers_)
print("Inertia: ", kmeans.inertia_)

In [ ]:
X_new = np.array([[0, 2], [3, 2], [-3, 1], [-3, 2.5]])
kmeans.predict(X_new)

##### Utility functions for plotting

In [ ]:
def plot_data(X):
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)

def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=30, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=50, linewidths=50,
                color=cross_color, zorder=11, alpha=1)

def plot_decision_boundaries(clusterer, X, resolution=1000, show_centroids=True,
                             show_xlabels=True, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                cmap="Pastel2")
    plt.contour(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                linewidths=1, colors='k')
    plot_data(X)
    if show_centroids:
        plot_centroids(clusterer.cluster_centers_)

    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
#KMeans producing a linear decision boundary
plt.figure(figsize=(12, 6))
plot_decision_boundaries(kmeans, X)
plt.show()

In [ ]:
def plot_clusterer_comparison(clusterer1, clusterer2, X, title1=None, title2=None):
    clusterer1.fit(X)
    clusterer2.fit(X)

    plt.figure(figsize=(12, 4))

    plt.subplot(121)
    plot_decision_boundaries(clusterer1, X)
    if title1:
        plt.title(title1, fontsize=14)

    plt.subplot(122)
    plot_decision_boundaries(clusterer2, X, show_ylabels=False)
    if title2:
        plt.title(title2, fontsize=14)

In [ ]:
kmeans_rnd_init1 = KMeans(n_clusters=5, init="random", n_init=1,
                         algorithm="full", random_state=11)
kmeans_rnd_init2 = KMeans(n_clusters=5, init="random", n_init=1,
                         algorithm="full", random_state=19)

plot_clusterer_comparison(kmeans_rnd_init1, kmeans_rnd_init2, X,
                          "Solution 1", "Solution 2 (with a different random init)")

plt.show()

In [ ]:
kmeans_rnd_init1 = KMeans(n_clusters=5, init="k-means++", n_init=1,
                         algorithm="full", random_state=11)
kmeans_rnd_init2 = KMeans(n_clusters=5, init="k-means++", n_init=1,
                         algorithm="full", random_state=19)

plot_clusterer_comparison(kmeans_rnd_init1, kmeans_rnd_init2, X,
                          "Solution 1", "Solution 2 (with a different random init)")

##### Cross validation to select the right K

In [ ]:
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(X)
                for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]

In [ ]:
plt.figure(figsize=(12, 5.5))
plt.plot(range(1, 10), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.annotate('Elbow',
             xy=(4, inertias[3]),
             xytext=(0.55, 0.55),
             textcoords='figure fraction',
             fontsize=16,
             arrowprops=dict(facecolor='black', shrink=0.1)
            )
plt.axis([1, 8.5, 0, 1300])
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_scores = [silhouette_score(X, model.labels_)
                     for model in kmeans_per_k[1:]]

plt.figure(figsize=(10, 4))
plt.plot(range(2, 10), silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.axis([1.8, 8.5, 0.55, 0.7])
plt.show()

In [ ]:
import matplotlib as mpl
from sklearn.metrics import silhouette_samples
from matplotlib.ticker import FixedLocator, FixedFormatter

plt.figure(figsize=(11, 9))

for k in (3, 4, 5, 6):
    plt.subplot(2, 2, k - 2)
    
    y_pred = kmeans_per_k[k - 1].labels_
    silhouette_coefficients = silhouette_samples(X, y_pred)

    print(len(silhouette_coefficients))
    print(type(silhouette_coefficients))
    print(silhouette_coefficients)
    padding = len(X) // 30
    pos = padding
    ticks = []
    for i in range(k):
        coeffs = silhouette_coefficients[y_pred == i]
        coeffs.sort()

        color = mpl.cm.Spectral(i / k)
        plt.fill_betweenx(np.arange(pos, pos + len(coeffs)), 0, coeffs,
                          facecolor=color, edgecolor=color, alpha=0.7)
        ticks.append(pos + len(coeffs) // 2)
        pos += len(coeffs) + padding

    plt.gca().yaxis.set_major_locator(FixedLocator(ticks))
    plt.gca().yaxis.set_major_formatter(FixedFormatter(range(k)))
    if k in (3, 5):
        plt.ylabel("Cluster")
    
    if k in (5, 6):
        plt.gca().set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
        plt.xlabel("Silhouette Coefficient")
    else:
        plt.tick_params(labelbottom=False)

    plt.axvline(x=silhouette_scores[k - 2], color="red", linestyle="--")
    plt.title("$k={}$".format(k), fontsize=16)

plt.show()

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,8))
for i in [2, 3, 4, 5]:
    km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42)
    q, mod = divmod(i, 2)
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(X) 

##### Poor man's Image segmentation

In [ ]:
from matplotlib.image import imread
from pylab import *

plt.figure(figsize=(8, 4))
image = imread("nature.jpg") 
imshow(image)
show()

print("Image Shape (3D Array): ", image.shape)
image.dtype

In [ ]:
X = image.reshape(-1, 3)
print("Dimension of Reshaped X: ", X.shape)

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=42).fit(X)

In [ ]:
print(f"Label count= {kmeans.labels_.shape}")
print("3D Cluster Center count=", kmeans.cluster_centers_.shape)
print("3D Cluster Centers:\n", kmeans.cluster_centers_)

In [ ]:
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(image)
plt.title("Original image")
plt.axis('off')

plt.subplot(122)
#plt.imshow(segmented_img/255)
plt.imshow((segmented_img * 255).astype(np.uint8))
plt.title("Segmented Image: {} colors".format(8))
show()

##### MNIST

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
X_digits, y_digits = load_digits(return_X_y=True)

print(X_digits.shape)

In [ ]:
some_digit = X_digits[16]
some_digit_image = some_digit.reshape(8, 8)
plt.imshow(some_digit_image, cmap = 'gray', interpolation="nearest")
plt.show()

In [ ]:
def visualize_digit(img):
    fig = plt.figure(figsize = (8,8)) 
    img = img.reshape(8, 8)
    plt.imshow(img, cmap='gray')
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            plt.annotate(str(round(img[x][y],2)), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y]<thresh else 'black')

visualize_digit(X_digits[16])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, random_state=42)
print(f"X_train dim {X_train.shape}")

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
lr.fit(X_train, y_train)

accuracy_before_preproc = lr.score(X_test, y_test)
print(f"Accuracy before preproc = {accuracy_before_preproc}")

##### KMeans for preprocessing

In [ ]:
# Step 1: Choose a number of clusters to be the projected dimension of our features and train the K-Means model.
kmeans = KMeans(n_clusters=50, random_state=42)
kmeans.fit(X_train)


# Step 2: Compute the distance of the data points from each cluster center. 
#         These distances will be the new feature vector.
X_train_new = kmeans.transform(X_train)
X_test_new = kmeans.transform(X_test)

print("Dimension of new features: ", X_train_new.shape)

# Classify the data using the new features
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train_new, y_train)


accuracy_after_preproc = log_reg.score(X_test_new, y_test)
print("Accuracy after preproc = ", accuracy_after_preproc)

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=50, random_state=42)),
    ("lr", LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)),
])
pipeline.fit(X_train, y_train)

accuracy_after_preproc = pipeline.score(X_test, y_test)

print(f"Accuracy after preproc = {accuracy_after_preproc}")

reduction_in_error = 1 - (1 - accuracy_after_preproc) / (1 - accuracy_before_preproc)
print("Reduction in Error Rate: %f" %  reduction_in_error)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=1, n_jobs=-1)
grid_clf.fit(X_train, y_train)

params_optimal_grid_clf = grid_clf.best_params_

print("Optimal Hyperparameter Values: ", params_optimal_grid_clf)
print("Grid Accuracy: ", grid_clf.score(X_test, y_test))

In [ ]:
print("Optimal Number of Clusters: ", params_optimal_grid_clf['kmeans__n_clusters'])

pipeline = Pipeline([
    ("kmeans", KMeans(n_clusters=params_optimal_grid_clf['kmeans__n_clusters'], random_state=42)),
    ("log_reg", LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)),
])
pipeline.fit(X_train, y_train)

accuracy_after_preproc = pipeline.score(X_test, y_test)

print("Accuracy after preproc: ", accuracy_after_preproc)

reduction_in_error = 1 - (1 - accuracy_after_preproc) / (1 - accuracy_before_preproc)
print("Reduction in Error Rate: %f" %  reduction_in_error)

##### Gaussian Mixture Models

In [ ]:
X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1 = X1.dot(np.array([[0.374, 0.95], [0.732, 0.598]]))
X2, y2 = make_blobs(n_samples=250, centers=1, random_state=42)
X2 = X2 + [6, -8]
X = np.r_[X1, X2]
y = np.r_[y1, y2]

In [ ]:
def plot_clusters(X, y=None):
    plt.scatter(X[:, 0], X[:, 1], c=y, s=5, cmap='autumn')
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0)
    
plt.figure(figsize=(12, 6))
plot_clusters(X)
plt.show()

In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gmm.fit(X)


In [ ]:
from matplotlib.colors import LogNorm
def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=30, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=50, linewidths=50,
                color=cross_color, zorder=11, alpha=1)


def plot_gaussian_mixture(clusterer, X, resolution=1000, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 norm=LogNorm(vmin=1.0, vmax=30.0),
                 levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z,
                norm=LogNorm(vmin=1.0, vmax=30.0),
                levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')

    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z,
                linewidths=2, colors='r', linestyles='dashed')
    
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
    plot_centroids(clusterer.means_, clusterer.weights_)

    plt.xlabel("$x_1$", fontsize=14)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
# Compute the log-likelihood of each sample
densities = gmm.score_samples(X)

# Define the threshold is at the fourth percentile lowest density
density_threshold = np.percentile(densities, 4) 

# Get the anomalous samples
anomalies = X[densities < density_threshold]

In [ ]:

plt.figure(figsize=(12, 6))
plt.title("Anomalies (red stars) Reside in the Low-Density Regions")

plot_gaussian_mixture(gmm, X)
plt.scatter(anomalies[:, 0], anomalies[:, 1], color='r', marker='*')
plt.ylim(top=5.1)

plt.show()

##### Decision boundary for mixture models

In [ ]:
X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1 = X1.dot(np.array([[0.374, 0.95], [0.732, 0.598]]))
X2, y2 = make_blobs(n_samples=250, centers=1, random_state=42)
X2 = X2 + [6, -8]
X = np.r_[X1, X2]
y = np.r_[y1, y2]


plt.figure(figsize=(12, 6))
plt.scatter(X[:, 0], X[:, 1], c=None, s=2, cmap='autumn')
plt.xlabel("$x_1$", fontsize=14)
plt.ylabel("$x_2$", fontsize=14, rotation=0)
plt.show()

In [ ]:
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(X)
                for k in range(1, 10)]



silhouette_scores = [silhouette_score(X, model.labels_)
                     for model in kmeans_per_k[1:]]

plt.figure(figsize=(10, 4))
plt.plot(range(2, 10), silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.show()

In [ ]:
# Number of clusters
k = 3

# Train the K-Means model
kmeans = KMeans(n_clusters=k, random_state=42, verbose=1)
kmeans.fit(X)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("Clustering by K-Means")
plot_decision_boundaries(kmeans, X)
plt.show()

In [ ]:
gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("Clustering by GMM")
plot_decision_boundaries(gm, X, show_centroids=False)
plt.show()

##### DBSCAN for non globular data

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.05, random_state=42)


plt.figure(figsize=(12, 6))
plt.scatter(X[:, 0], X[:, 1], c=None, s=5, cmap='autumn')
plt.title("Two Arbitrary Shape Clusters", fontsize=16)
plt.xlabel("$x_1$", fontsize=14)
plt.ylabel("$x_2$", fontsize=14, rotation=0)
plt.show()

In [ ]:
# Number of clusters
k = 2

# Train the K-Means model
kmeans = KMeans(n_clusters=k, random_state=42, verbose=1)
kmeans.fit(X)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("Clustering by K-Means")
plot_decision_boundaries(kmeans, X)
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.15, min_samples=5)
dbscan.fit(X)

In [ ]:
def plot_dbscan(dbscan, X, size, show_xlabels=True, show_ylabels=True):
    core_mask = np.zeros_like(dbscan.labels_, dtype=bool)
    core_mask[dbscan.core_sample_indices_] = True
    anomalies_mask = dbscan.labels_ == -1
    non_core_mask = ~(core_mask | anomalies_mask)

    cores = dbscan.components_
    anomalies = X[anomalies_mask]
    non_cores = X[non_core_mask]
    
    plt.scatter(cores[:, 0], cores[:, 1],
                c=dbscan.labels_[core_mask], marker='o', s=size, cmap="Paired")
    plt.scatter(cores[:, 0], cores[:, 1], marker='*', s=20, c=dbscan.labels_[core_mask])
    plt.scatter(anomalies[:, 0], anomalies[:, 1],
                c="r", marker="x", s=100)
    plt.scatter(non_cores[:, 0], non_cores[:, 1], c=dbscan.labels_[non_core_mask], marker=".")
    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)
    plt.title("Clustering by DBSCAN\neps={:.2f}, min_samples={}".format(dbscan.eps, dbscan.min_samples), fontsize=14)

In [ ]:
plt.figure(figsize=(12, 6))
plot_dbscan(dbscan, X, size=100)
plt.show()